In [2]:
import pandas as pd
pd.set_option("display.max_rows", None, "display.max_columns", None)#to display complete dataframe
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import linear_model
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error 

### Function for linear regression imputation

In [7]:
#This function fills the null values of the columns which has high correlation with a column
#which does not have null values
def regression_imputer(dataframe, column_name):
    #dataframe must has a column called 'count' which is only a consecutive number of rows.
    df = dataframe.copy()  
    #regression between count and 'column_name'
    #The following code generates a df with the neccesary data to train the regression model, 
    #excluding the null values.
    model_data = df[['count', f'{column_name}']][df[f'{column_name}'].notnull()]
    #create x and y arrays, we must reshape it because LinearRegression() object only accepts array 
    #like inputs
    x = model_data['count'].values.reshape(-1, 1)
    y = model_data[f'{column_name}'].values.reshape(-1, 1)
    #Create the Linear Regression model to estimate the missing data
    regression_model = linear_model.LinearRegression()
    #fit the model
    regression_model.fit(X = x, y = y)
    #generate the x values to be predicted (the 'count' values), the following line extracts from 
    #the 'count' column the values where in 'column_name' are nulls, so it extract the X's needed to
    #be preddicted. 
    x_to_predict = df['count'][df[f'{column_name}'].isnull()].values.reshape(-1,1)
    #Make the predictions
    predictions = regression_model.predict(x_to_predict)
    #We need to transform the predictions array into a numeric list.
    predictions = [float(i) for i in predictions] 
    #From the dataframe, select all the null positions in the column, and then replace them by the 
    #new predictions
    df[f'{column_name}'][df[f'{column_name}'].isnull()] = predictions
    return df

In [3]:
#Read the 'indice de cartera vencida' file, and set the col 0 ('Fecha') be the dataframe index.
indice_df = pd.read_excel('icv_mensual.xlsx', index_col=0)

#Resample the ICV_cartera_total into yearly periods, so we get the average icv of each year 
yearly_resampled_indice_df = indice_df.ICV_cartera_total.resample('Y').mean()

In [19]:
yearly_resampled_indice_df

Fecha
2002-12-31    0.099150
2003-12-31    0.079888
2004-12-31    0.053555
2005-12-31    0.032498
2006-12-31    0.028531
2007-12-31    0.030686
2008-12-31    0.040477
2009-12-31    0.046378
2010-12-31    0.039239
2011-12-31    0.029018
2012-12-31    0.028363
2013-12-31    0.030456
2014-12-31    0.030746
2015-12-31    0.029301
2016-12-31    0.032214
2017-12-31    0.042261
2018-12-31    0.048683
2019-12-31    0.046524
2020-12-31    0.044792
Freq: A-DEC, Name: ICV_cartera_total, dtype: float64

In [18]:
len(yearly_resampled_indice_df)

19

### Resample the variables dataset

In [5]:
#read the 'variables' dataset
variables_df = pd.read_excel('variables_macro_trimestral.xlsx', index_col=0)
#add an incremental column number to see how the variables change with transcurred time
variables_df['count'] = range(0, len(variables_df.index))

In [8]:
#The following list takes the column names which are going to be imputed with regression technique.
columns_to_regression_imputer = ['IPC', 'PIB']

#Create a copy of the dataframe with the null columns
imputed_df = variables_df.copy()

#apply the regression imputer to the columns that need it 
for column in columns_to_regression_imputer:

    imputed_df = regression_imputer(imputed_df, column)

c:\Users\HP\Anaconda3\lib\site-packages\ipykernel_launcher.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
c:\Users\HP\Anaconda3\lib\site-packages\ipykernel_launcher.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [9]:
imputed_df

,Desempleo,IPC,TRM,Exportaciones,Importaciones,PIB,count
Fecha,,,,,,,
2001-03-01,0.200207,44.307221,2278.780000,NaN,NaN,105742.683730,0
2001-06-01,0.181627,45.076797,2305.660000,NaN,NaN,107281.144141,1
2001-09-01,0.177623,45.846372,2328.230000,NaN,NaN,108819.604551,2
2001-12-01,0.166253,46.615947,2306.900000,NaN,NaN,110358.064962,3
2002-03-01,0.190028,47.385523,2282.330000,NaN,NaN,111896.525372,4
2002-06-01,0.177497,48.155098,2364.250000,NaN,NaN,113434.985782,5
2002-09-01,0.179465,48.924673,2751.230000,NaN,NaN,114973.446193,6
2002-12-01,0.157456,49.694249,2814.890000,NaN,NaN,116511.906603,7
2003-03-01,0.177929,51.510000,2959.010000,NaN,NaN,118050.367014,8


In [15]:
#Resample the variables into yearly periods
yearly_imputed_df = imputed_df.resample('Y').mean()
yearly_imputed_df.drop('count', inplace=True, axis=1)
yearly_imputed_df['count'] = range(0, len(yearly_imputed_df.index))

In [22]:
yearly_imputed_df

,Desempleo,IPC,TRM,Exportaciones,Importaciones,PIB,count
Fecha,,,,,,,
2001-12-31,0.181428,45.461584,2304.892500,NaN,NaN,108050.374346,0
2002-12-31,0.176112,48.539886,2553.175000,NaN,NaN,114204.215988,1
2003-12-31,0.165934,52.360000,2858.310000,NaN,NaN,120358.057629,2
2004-12-31,0.153036,55.470000,2587.877500,NaN,NaN,126511.899271,3
2005-12-31,0.139230,58.200000,2314.732500,NaN,NaN,128713.250000,4
2006-12-31,0.129204,60.695000,2366.203816,NaN,NaN,137358.750000,5
2007-12-31,0.113842,64.107500,2064.100375,NaN,NaN,146614.250000,6
2008-12-31,0.115381,68.657500,1969.485000,NaN,NaN,151428.250000,7
2009-12-31,0.130004,71.245000,2141.267500,8.211582e+06,8.222783e+06,153154.000000,8


In [39]:
predictores_icv_2021 = yearly_imputed_df.iloc[-1].values.reshape(-1, 7)
yearly_imputed_df = yearly_imputed_df.iloc[:-1]

#### Set aside the test dataset

In [24]:
#Save the predictors in 'x' and the labels in 'y'
x = yearly_imputed_df
y = list(yearly_resampled_indice_df)

#create the train and test data
from sklearn.model_selection import train_test_split
train_x, test_x, train_y, test_y = train_test_split(x, y, random_state = 42)

In [25]:
#Let's impute the columns 'Exportaciones' and 'Importaciones' with their respective means
from sklearn.impute import SimpleImputer
imputer = SimpleImputer()
imputed_train_x = pd.DataFrame(imputer.fit_transform(train_x))
imputed_test_x = pd.DataFrame(imputer.transform(test_x)) #Only we use .transform() because this is 
#out_sample data.

#imputation removed col names, put them back
imputed_train_x.columns = train_x.columns
imputed_test_x.columns = test_x.columns

### Modeling yearly with Random Forest

In [30]:
#RANDOM FOREST
model = RandomForestRegressor(random_state=1)
model.fit(imputed_train_x, train_y)

predictions = model.predict(imputed_test_x)
print(mean_absolute_error(test_y, predictions))

0.022005052707392704


#### Predicting the icv for the 2021 year

In [46]:
prediction_for_2021 = model.predict(predictores_icv_2021)
print(prediction_for_2021)

[0.04638018]
